# 2. Создание нейронной сети без использования готовых решений

__Автор__: Никита Владимирович Блохин (NVBlokhin@fa.ru)

Финансовый университет, 2020 г. 

In [322]:
import torch
import torch.nn as nn
import torch.nn.functional as F

In [323]:
torch.set_warn_always(True)

## 1. Создание нейронов и полносвязных слоев

1.1. Используя операции над матрицами и векторами из библиотеки `torch`, реализовать нейрон с заданными весами `weights` и `bias`. Прогнать вектор `inputs` через нейрон и вывести результат. 

In [12]:
class Neuron:

    def __init__(self, weights: torch.Tensor, bias: torch.Tensor):
        self.weights = weights
        self.bias = bias

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.sum(inputs * self.weights) + self.bias

In [13]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([-0.2, 0.3, -0.5, 0.7])
bias = torch.tensor(3.14)

In [14]:
neuron = Neuron(weights, bias)
print(f'Neuron.forward: {neuron.forward(inputs)}')
print(f'functional.linear: {F.linear(inputs, weights, bias)}')

Neuron.forward: 4.840000152587891
functional.linear: 4.840000152587891


1.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать полносвязный слой с заданными весами `weights` и `biases`. Прогнать вектор `inputs` через слой и вывести результат. 

In [15]:
class Linear:

    def __init__(self, weights: torch.Tensor, biases: torch.Tensor):
        self.weights = weights
        self.biases = biases

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.matmul(inputs, self.weights.T) + self.biases

In [16]:
inputs = torch.tensor([1.0, 2.0, 3.0, 4.0])
weights = torch.tensor([[-0.2, 0.3, -0.5, 0.7],
                        [0.5, -0.91, 0.26, -0.5],
                        [-0.26, -0.27, 0.17, 0.87]])  # убрал .T

biases = torch.tensor([3.14, 2.71, 7.2])

In [17]:
m = Linear(weights, biases)
print(f'Linear.forward:\n{m.forward(inputs)}')
print(f'\nfunctional.linear:\n{F.linear(inputs, weights, biases)}')

Linear.forward:
tensor([ 4.8400,  0.1700, 10.3900])

functional.linear:
tensor([ 4.8400,  0.1700, 10.3900])


1.3 Реализовать полносвязный слой из __2.1.2__ таким образом, чтобы он мог принимать на вход матрицу (батч) с данными. Продемонстрировать работу.
Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.


In [18]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

In [19]:
m = Linear(weights, biases)
print(f'Linear.forward:\n{m.forward(inputs)}')
print(f'\nfunctional.linear:\n{F.linear(inputs, weights, biases)}')

Linear.forward:
tensor([[ 3.7900,  0.9200,  9.0850],
        [ 6.1400, -2.1000,  6.9000],
        [ 2.0400,  0.7610,  6.7260]])

functional.linear:
tensor([[ 3.7900,  0.9200,  9.0850],
        [ 6.1400, -2.1000,  6.9000],
        [ 2.0400,  0.7610,  6.7260]])


1.4 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать полносвязный слой из `n_neurons` нейронов с `n_features` весами у каждого нейрона (инициализируются из стандартного нормального распределения). Прогнать вектор `inputs` через слой и вывести результат. Результатом прогона сквозь слой должна быть матрица размера `batch_size` x `n_neurons`.

In [278]:
class Linear:

    def __init__(self, in_features: int, out_features: int):
        self.in_features = in_features
        self.out_features = out_features

        self.weights = torch.randn(out_features, in_features, requires_grad=True)
        self.biases = torch.randn(out_features, requires_grad=True)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.matmul(inputs, self.weights.T) + self.biases

In [22]:
torch.manual_seed(0)

m = Linear(4, 3)
m.forward(inputs)

tensor([[ -5.0178,   0.5240,  -3.9319],
        [  4.0738,  -6.7887,  -3.5657],
        [-11.6052,  -0.3882,  -3.1959]])

In [23]:
# проверка
nn_m = nn.Linear(4, 3)
nn_m.weight = nn.Parameter(m.weights)
nn_m.bias = nn.Parameter(m.biases)
nn_m.forward(inputs)

tensor([[ -5.0178,   0.5240,  -3.9319],
        [  4.0738,  -6.7887,  -3.5657],
        [-11.6052,  -0.3882,  -3.1959]], grad_fn=<AddmmBackward0>)

1.5 Используя решение из __1.4__, создать 2 полносвязных слоя и пропустить матрицу `inputs` последовательно через эти два слоя. Количество нейронов в первом слое выбрать произвольно, количество нейронов во втором слое выбрать так, чтобы результатом прогона являлась матрица (3x7).

In [39]:
class NeuralNet:

    def __init__(self):
        self.input = Linear(4, 3)
        self.output = Linear(3, 7)

    def forward(self, inputs) -> torch.Tensor:
        return self.output.forward(self.input.forward(inputs))

In [40]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

In [41]:
torch.manual_seed(0)

m = NeuralNet()
m.forward(inputs)

tensor([[  5.1553,   1.1916,  -4.7817,   3.6611,  -9.6546,   0.1692,   5.8099],
        [ -0.8347,   0.0913,   3.1227,   3.5542,   5.4400,   4.9157,   8.2533],
        [  8.8057,  -1.1342,  -7.8488,   6.7415, -11.1044,   5.1500,   5.6152]])

## 2. Создание функций активации

In [77]:
def randn(*size: int, seed: int = 0) -> torch.Tensor:
    torch.manual_seed(seed)
    return torch.randn(*size)

2.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию активации ReLU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/f4353f4e3e484130504049599d2e7b040793e1eb)

Создать матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверить работоспособность функции активации.

In [78]:
class ReLU:

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.maximum(inputs, torch.tensor(0))

In [79]:
inputs = randn(4, 3)
inputs

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986],
        [ 0.4033,  0.8380, -0.7193],
        [-0.4033, -0.5966,  0.1820]])

In [80]:
relu = ReLU()
nn_relu = nn.ReLU()
print(f'ReLU:\n{relu.forward(inputs)}')
print(f'\nnn.ReLU:\n{nn_relu.forward(inputs)}')

ReLU:
tensor([[1.5410, 0.0000, 0.0000],
        [0.5684, 0.0000, 0.0000],
        [0.4033, 0.8380, 0.0000],
        [0.0000, 0.0000, 0.1820]])

nn.ReLU:
tensor([[1.5410, 0.0000, 0.0000],
        [0.5684, 0.0000, 0.0000],
        [0.4033, 0.8380, 0.0000],
        [0.0000, 0.0000, 0.1820]])


2.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию активации softmax:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/6d7500d980c313da83e4117da701bf7c8f1982f5)

Создать матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверить работоспособность функции активации. Строки матрицы трактовать как выходы линейного слоя некоторого классификатора для 4 различных примеров.

In [100]:
class Softmax:

    def __init__(self, dim: int = 0):
        assert dim == 0 or dim == 1
        self.dim = dim

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        exp = torch.exp(inputs)
        return exp / torch.sum(exp, dim=self.dim).unsqueeze(self.dim)

In [101]:
inputs = randn(4, 3)
inputs

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986],
        [ 0.4033,  0.8380, -0.7193],
        [-0.4033, -0.5966,  0.1820]])

In [102]:
softmax = Softmax(dim=1)
nn_softmax = nn.Softmax(dim=1)
print(f'Softmax:\n{softmax.forward(inputs)}')
print(f'\nnn.Softmax:\n{nn_softmax.forward(inputs)}')

Softmax:
tensor([[0.8446, 0.1349, 0.0205],
        [0.7511, 0.1438, 0.1051],
        [0.3484, 0.5382, 0.1134],
        [0.2762, 0.2277, 0.4961]])

nn.Softmax:
tensor([[0.8446, 0.1349, 0.0205],
        [0.7511, 0.1438, 0.1051],
        [0.3484, 0.5382, 0.1134],
        [0.2762, 0.2277, 0.4961]])


2.3 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию активации ELU:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/eb23becd37c3602c4838e53f532163279192e4fd)

Создать матрицу размера (4,3), заполненную числами из стандартного нормального распределения, и проверить работоспособность функции активации.

In [107]:
class ELU:

    def __init__(self, alpha: float = 1):
        self.alpha = alpha

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.where(inputs > 0, inputs, self.alpha * (torch.exp(inputs) - 1))

In [108]:
inputs = randn(4, 3)
inputs

tensor([[ 1.5410, -0.2934, -2.1788],
        [ 0.5684, -1.0845, -1.3986],
        [ 0.4033,  0.8380, -0.7193],
        [-0.4033, -0.5966,  0.1820]])

In [110]:
elu = ELU(alpha=1.409)
nn_elu = nn.ELU(alpha=1.409)
print(f'ELU:\n{elu.forward(inputs)}')
print(f'\nnn.ELU:\n{nn_elu.forward(inputs)}')

ELU:
tensor([[ 1.5410, -0.3583, -1.2495],
        [ 0.5684, -0.9327, -1.0611],
        [ 0.4033,  0.8380, -0.7227],
        [-0.4677, -0.6331,  0.1820]])

nn.ELU:
tensor([[ 1.5410, -0.3583, -1.2495],
        [ 0.5684, -0.9327, -1.0611],
        [ 0.4033,  0.8380, -0.7227],
        [-0.4677, -0.6331,  0.1820]])


## 3. Создание функции потерь

3.1 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию потерь MSE:

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/e258221518869aa1c6561bb75b99476c4734108e)

Создать полносвязный слой с 1 нейроном, прогнать через него батч `inputs` и посчитать значение MSE, трактуя вектор `y` как вектор правильных ответов.

In [118]:
class MSELoss:

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return torch.mean((y_true - y_pred) ** 2)

In [119]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

y = torch.tensor([2, 3, 4]).unsqueeze(1)

In [121]:
torch.manual_seed(0)

layer = Linear(4, 1)
y_pred = layer.forward(inputs)

In [122]:
mse = MSELoss()
nn_mse = nn.MSELoss()
print(f'MSELoss:\n{mse.forward(y_pred, y)}')
print(f'\nnn.MSELoss:\n{nn_mse.forward(y_pred, y)}')

MSELoss:
101.30004119873047

nn.MSELoss:
101.30004119873047


3.2 Используя операции над матрицами и векторами из библиотеки `torch`, реализовать функцию потерь Categorical Cross-Entropy:

<img src="https://i.ibb.co/93gy1dN/Screenshot-9.png" width="200">

Создать полносвязный слой с 3 нейронами и прогнать через него батч `inputs`. Полученный результат пропустить через функцию активации softmax. Посчитать значение CCE, трактуя вектор `y` как вектор правильных ответов.

In [174]:
class CategoricalCrossEntropyLoss:

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return -torch.sum(y_true * torch.log(y_pred), dim=1)

In [175]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

y = torch.tensor([1, 0, 0])

In [176]:
torch.manual_seed(0)

layer = Linear(4, 3)
softmax = Softmax()
y_pred = softmax.forward(layer.forward(inputs))

In [177]:
cce = CategoricalCrossEntropyLoss()
cce.forward(y_pred, y)

tensor([9.0918e+00, 1.1272e-04, 1.5679e+01])

3.3 Модифицировать 2.3.1, добавив L2-регуляризацию.

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/d92ca2429275bfdc0474523babbafe014ca8b580)


In [196]:
class MSELossL2:

    def __init__(self, lambda_: float, weights: torch.Tensor):
        self.lambda_ = lambda_
        self.weights = weights

    def data_loss(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return torch.sum((y_true - y_pred) ** 2)

    def reg_loss(self) -> torch.Tensor:
        return self.lambda_ * torch.sum(self.weights ** 2)

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return self.data_loss(y_pred, y_true) + self.reg_loss()

In [197]:
inputs = torch.tensor([[1, 2, 3, 2.5],
                       [2, 5, -1, 2],
                       [-1.5, 2.7, 3.3, -0.8]])

y = torch.tensor([2, 3, 4]).unsqueeze(1)

In [198]:
torch.manual_seed(0)

layer = Linear(4, 1)
y_pred = layer.forward(inputs)

In [199]:
mse_l2 = MSELossL2(lambda_=1.409, weights=layer.weights)
mse_l2.forward(y_pred, y)

tensor(314.5113)

## 4. Обратное распространение ошибки

4.1 Используя один нейрон и SGD (1 пример за шаг), решите задачу регрессии

In [219]:
from sklearn.datasets import make_regression

X, y, coef = make_regression(n_features=4, n_informative=4, coef=True, bias=0.5)
# X.dtype == float64 - нет смысла использовать torch.from_numpy
# т.к. последующее приведение типов Tensor.type(torch.float32) приведет к копированию данных
X = torch.tensor(X, dtype=torch.float32)
y = torch.tensor(y, dtype=torch.float32)

[Граф вычислений для этой задачи](https://i.ibb.co/2dhDxZx/photo-2021-02-15-17-18-04.jpg)

4.1.1 Модифицируйте класс `MSELoss` из __2.3.1__, реализовав расчет производной относительно предыдущего слоя


In [234]:
class MSELoss:

    def forward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        return torch.mean((y_pred - y_true) ** 2)

    def backward(self, y_pred: torch.Tensor, y_true: torch.Tensor) -> torch.Tensor:
        e = y_pred - y_true
        return 2 * e  # df/dc

4.1.2. Модифицируйте класс `Neuron` из __2.1.1__:

  1) Сделайте так, чтобы веса нейрона инициализировались из стандартного нормального распределения

  2) Реализуйте расчет градиента относительно весов `weights` и `bias`

In [241]:
class Neuron:

    def __init__(self, in_features: int):
        self.in_features = in_features
        self.weights = torch.randn(self.in_features)
        self.bias = torch.randn(1)

    def forward(self, inputs: torch.Tensor) -> torch.Tensor:
        return torch.sum(inputs * self.weights) + self.bias

    def backward(self, dvalue: torch.Tensor) -> tuple[torch.Tensor, torch.Tensor]:
        dweights = dvalue * 1  # df/dw
        dbias = dvalue * 1  # df/db
        dinputs = dbias * self.weights  # df/dx зачем?
        return dweights, dbias

4.1.3 Допишите цикл для настройки весов нейрона

[SGD](https://ru.wikipedia.org/wiki/%D0%A1%D1%82%D0%BE%D1%85%D0%B0%D1%81%D1%82%D0%B8%D1%87%D0%B5%D1%81%D0%BA%D0%B8%D0%B9_%D0%B3%D1%80%D0%B0%D0%B4%D0%B8%D0%B5%D0%BD%D1%82%D0%BD%D1%8B%D0%B9_%D1%81%D0%BF%D1%83%D1%81%D0%BA)

![](https://wikimedia.org/api/rest_v1/media/math/render/svg/dda3670f8a8996a0d3bf80856bb4a166cc8db6d4)

In [304]:
n_inputs = X.size(1)  # размерность элемента выборки
learning_rate = 0.1  #  скорость обучения
n_epoch = 100  #  количество эпох

neuron = Neuron(n_inputs)
loss = MSELoss()

losses = []
for epoch in range(n_epoch):
    for x, y_true in zip(X, y):  # а это точно нужно делать во внутреннем цикле?
        # forward pass
        y_pred = neuron.forward(x)
        curr_loss = loss.forward(y_pred, y_true)
        losses.append(curr_loss)

        # backprop
        dw, db = neuron.backward(loss.backward(y_pred, y_true))

        neuron.weights -= learning_rate * dw
        neuron.bias -= learning_rate * db

In [305]:
# работает как часы 🕓... (нет)
losses[:5], losses[-5:], neuron.weights, coef

([tensor(13452.7949),
  tensor(3068.4236),
  tensor(2568.9526),
  tensor(7889.0342),
  tensor(2024.4634)],
 [tensor(23572.7812),
  tensor(38770.8945),
  tensor(66.2022),
  tensor(16.8464),
  tensor(786.2955)],
 tensor([38.4900, 34.9740, 35.9676, 36.3822]),
 array([75.61322997,  0.70372677, 91.67887498, 43.2366121 ]))

In [308]:
learning_rate = 0.1  # скорость обучения
n_epoch = 100  # количество эпох

layer = Linear(X.size(1), 1)
mse = MSELoss()

for epoch in range(1, n_epoch + 1):
    y_pred = layer.forward(X)
    loss = mse.forward(y_pred, y)

    loss.backward()
    with torch.no_grad():
        layer.weights -= learning_rate * layer.weights.grad
        layer.biases -= learning_rate * layer.biases.grad

    if epoch % 10 == 0:
        print(f'epoch {epoch}:\n\tdw = {layer.weights.grad}\n\tw = {layer.weights}\n\tloss = {loss}\n')

    layer.weights.grad.zero_()
    layer.biases.grad.zero_()

epoch 10:
	dw = tensor([[-0.7804, -0.1373,  0.7393,  0.0561]])
	w = tensor([[-0.4865, -0.1190,  0.8135, -0.0180]], requires_grad=True)
	loss = 14425.232421875

epoch 20:
	dw = tensor([[-0.1754, -0.0499,  0.3540, -0.0156]])
	w = tensor([[-0.1014, -0.0342,  0.2397, -0.0151]], requires_grad=True)
	loss = 14421.5244140625

epoch 30:
	dw = tensor([[-0.0354, -0.0130,  0.0902, -0.0064]])
	w = tensor([[-0.0206, -0.0080,  0.0564, -0.0045]], requires_grad=True)
	loss = 14421.390625

epoch 40:
	dw = tensor([[-0.0072, -0.0029,  0.0205, -0.0017]])
	w = tensor([[-0.0043, -0.0017,  0.0125, -0.0011]], requires_grad=True)
	loss = 14421.3857421875

epoch 50:
	dw = tensor([[-0.0015, -0.0006,  0.0045, -0.0004]])
	w = tensor([[-0.0009, -0.0004,  0.0027, -0.0003]], requires_grad=True)
	loss = 14421.3857421875

epoch 60:
	dw = tensor([[-3.1723e-04, -1.2780e-04,  9.7968e-04, -9.0900e-05]])
	w = tensor([[-1.8959e-04, -7.4979e-05,  5.9280e-04, -5.5773e-05]],
       requires_grad=True)
	loss = 14421.3837890625



In [343]:
# это вот работает
Y = y.unsqueeze(1)

layer = Linear(X.size(1), 1)
mse = nn.MSELoss()

learning_rate = 0.01409
optimizer = torch.optim.SGD([layer.weights, layer.biases], lr=learning_rate)

epochs = 500
for epoch in range(1, epochs + 1):
    y_pred = layer.forward(X)
    loss = mse.forward(y_pred, Y)

    loss.backward()
    optimizer.step()

    if epoch % 10 == 0:
        print(f'epoch {epoch}:'
              f'\n\tdw = {layer.weights.grad}\n\tdb = {layer.biases.grad}\n\t'
              f'w = {layer.weights}\n\tb = {layer.biases}\n\t'
              f'loss = {loss}\n')

    optimizer.zero_grad()


epoch 10:
	dw = tensor([[-116.7529,   20.9540, -126.0042,  -52.0535]])
	db = tensor([-17.5067])
	w = tensor([[19.0080, -2.5065, 22.0046,  8.7914]], requires_grad=True)
	b = tensor([2.2568], requires_grad=True)
	loss = 8773.7451171875

epoch 20:
	dw = tensor([[-86.7476,  10.1422, -97.0013, -42.5946]])
	db = tensor([-9.5114])
	w = tensor([[33.0256, -4.5637, 37.4192, 15.3730]], requires_grad=True)
	b = tensor([4.0662], requires_grad=True)
	loss = 5162.01171875

epoch 30:
	dw = tensor([[-64.6843,   3.4048, -74.8942, -34.7179]])
	db = tensor([-4.2518])
	w = tensor([[43.4603, -5.4323, 49.3041, 20.7471]], requires_grad=True)
	b = tensor([4.9724], requires_grad=True)
	loss = 3074.840576171875

epoch 40:
	dw = tensor([[-48.4055,  -0.6268, -57.9959, -28.2039]])
	db = tensor([-0.8927])
	w = tensor([[51.2556, -5.5744, 58.4948, 25.1195]], requires_grad=True)
	b = tensor([5.2929], requires_grad=True)
	loss = 1852.9224853515625

epoch 50:
	dw = tensor([[-36.3526,  -2.8867, -45.0419, -22.8474]])
	db =

4.2 Решите задачу 2.4.1, используя пакетный градиентный спуск

Вычисления для этой задачи: 
[1](https://i.ibb.co/rmtQT6P/photo-2021-02-15-18-00-43.jpg)
[2](https://i.ibb.co/NmCFVnQ/photo-2021-02-15-18-01-17.jpg)

4.2.1 Модифицируйте класс `MSELoss` из __3.1__, реализовав расчет производной относительно предыдущего слоя с учетом того, что теперь работа ведется с батчами, а не с индивидуальными примерами
 

In [ ]:
class MSELoss:
    def forward(self, y_pred, y_true):
        return  # <реализовать логику MSE>

    def backward(self, y_pred, y_true):
        self.dinput =  # df/dy^


4.2.2. Модифицируйте класс `Neuron` из __4.1.2__:

  1) Реализуйте метод `forward` таким образом, чтобы он мог принимать на вход матрицу (батч) с данными. 

  2) Реализуйте расчет градиента относительно весов `weights` и `bias` с учетом того, что теперь работа ведется с батчами, а не с индивидуальными примерами

In [ ]:
class Neuron:
    def __init__(self, n_inputs):
        # <создать атрибуты объекта weights и bias>
        pass

    def forward(self, inputs):
        return  # <реализовать логику нейрона>

    def backward(self, dvalue):
        # dvalue - значение градиента, которое приходит нейрону от следующего слоя сети
        # в данном случае это будет градиент L по y^ (созданный методом backwards у объекта MSELoss)
        self.dweights =  # df/dW
        self.dbias =  # df/db


4.2.3 Допишите цикл для настройки весов нейрона

In [ ]:
n_inputs =  # <размерность элемента выборки >
learning_rate = 0.1  #  скорость обучения
n_epoch = 100  #  количество эпох

neuron = Neuron(n_inputs)
loss = MSELoss()

for epoch in range(100):
    # forward pass
    y_pred =  # <прогон через нейрон>
    curr_loss =  # <прогон через функцию потерь>
    losses.append(curr_loss)

    # backprop
    # <вызов методов backward>
    # обратите внимание на последовательность вызовов: от конца к началу

    # <шаг оптимизации для весов (weights и bias) нейрона>

4.3  Используя один полносвязный слой и  пакетный градиетный спуск, решите задачу регрессии из __2.4.1__

4.3.1 Модифицируйте класс `Linear` из __1.4__. ([вычисление градиентов](https://i.ibb.co/kgVR6m6/photo-2021-02-15-21-30-28.jpg))

In [ ]:
class Linear:
    def __init__(self, n_features, n_neurons):
        # <создать атрибуты объекта weights и biases>
        pass

    def forward(self, inputs):
        return  # <реализовать логику слоя>

    def backward(self, dvalues):
        self.dweights =  # df/dW
        self.dbiases =  # df/db
        self.dinputs =  # df/dX

4.3.2 Создайте слой с одним нейроном. Используя класс MSELoss из 2.4.2, убедитесь, что модель обучается

4.4 Используя наработки из 2.4, создайте нейросеть и решите задачу регрессии.

Предлагаемая архитектура: 
1. Полносвязный слой с 10 нейронами
2. Активация ReLU
3. Полносвязный слой с 1 нейроном

In [ ]:
X = torch.linspace(-1, 1, 100).view(-1, 1)
y = X.pow(2) + 0.2 * torch.rand(X.size())

In [ ]:
class Activation_ReLU:
    def forward(self, inputs):
        self.inputs = inputs
        self.output = inputs.clip(min=0)
        return self.output

    def backward(self, dvalues):
        self.dinputs = dvalues.clone()
        self.dinputs[self.inputs <= 0] = 0

In [ ]:
# создание компонентов сети
# fc1 = 
# relu1 = 
# fc2 = 

loss = MSELoss()
lr = 0.02

ys = []
for epoch in range(2001):
    # <forward pass>
    # fc1 > relu1 > fc2 > loss

    data_loss =  # <прогон через функцию потерь>

    if epoch % 200 == 0:
        print(f'epoch {epoch} mean loss {data_loss}')
        ys.append(out)

    # <backprop>
    # loss > fc2 > relu1 > fc1

    # <шаг оптимизации для fc1>

    # <шаг оптимизации для fc2>


In [ ]:
import matplotlib.pyplot as plt

fig, axs = plt.subplots(len(ys), 1, figsize=(10, 40))
for ax, y_ in zip(axs, ys):
    ax.scatter(X.numpy(), y.numpy(), color="orange")
    ax.plot(X.numpy(), y_.numpy(), 'g-', lw=3)
    ax.set_xlim(-1.05, 1.5)
    ax.set_ylim(-0.25, 1.25)